<a href="https://colab.research.google.com/github/keyong2523/Will-you-talk-with-me/blob/main/kobert_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')

# 학습용 데이터셋 불러오기
import pandas as pd
dataset_train1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/자연어처리/user_conversation.csv')
dataset_train1.head()

# 데이터 전처리
dataset_train1.drop(['Unnamed: 0', 'Unnamed: 0.1', '질문 제목', '작성 시간', '태그', 'url'], axis=1, inplace=True)
dataset_train1.head()

dataset_train1['질병명'].unique()

# 대표 질병 5개 추출
data1 = dataset_train1[dataset_train1['질병명'] == '수족냉증']
data2 = dataset_train1[dataset_train1['질병명'] == '식중독']
data3 = dataset_train1[dataset_train1['질병명'] == '소화불량']
data4 = dataset_train1[dataset_train1['질병명'] == '질염']
data5 = dataset_train1[dataset_train1['질병명'] == '비염']
new_data = data1.append([data2, data3, data4, data5], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

# 질병명 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['질병명'])
new_data['질병명'] = encoder.transform(new_data['질병명'])
new_data.head()

# 라벨링된 질병명 매핑 ex) {0: '비염', 1: '소화불량', 2: '수족냉증', 3: '식중독', 4: '질염'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))
'''


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 
!pip install torch

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 27.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595735 sha256=f3de1eaf057711830b6e640e4c8d002be32f5abeb364b22e94f0ef7be112b0af
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 31.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.p

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
##GPU 사용 시
device = torch.device("cuda:0")

bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
train_df = pd.read_csv('emotion_Training.csv')
train_df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,44164.0,청년,남성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,아내가 드디어 출산하게 되어서 정말 신이 나.,아내분이 출산을 하시는군요. 정말 축하드려요.,아 지금 정말 신이 나.,잘 된 일이네요.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,좋은 아빠가 되실 거 같아요. 진심으로 축하드려요.,NaN,NaN
1,3926.0,노년,남성,"건강, 죽음",만성질환 유,불안,스트레스 받는,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,약 종류가 많아 번거로우시겠어요.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,현재 상황에서 변화를 주기 위해 어떻게 하면 될까요?,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,약을 잘 챙겨 먹어 건강해지시길 바랄게요.,NaN,NaN
2,50882.0,청소년,여성,학업 및 진로,해당없음,당황,당황,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,고등학교 수업이 중학교 때와는 다르게 갑자기 어려워서 당황스러우시군요.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,이 상황에서 할 수 있는 게 어떤 것이 있을까요?,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,수업을 어떤 방식으로 열심히 듣는게 좋을까요?,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.,열심히 해서 좋은 결과가 있었으면 좋겠어요.
3,31303.0,노년,남성,재정,만성질환 무,기쁨,신이 난,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,재취업 후 첫 월급이라 정말 기쁘시겠어요.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,이 기분을 유지하려면 어떻게 해야 할까요?,회사생활을 열심히 해서 계속 월급을 받을거야!,계속 원하시는 대로 회사생활이 이어지길 바랄게요.,NaN,NaN
4,47200.0,노년,여성,재정,만성질환 유,기쁨,안도,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,기분 좋으시겠어요. 앞으로는 어떻게 하실 생각이신가요?,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶으시군요.,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69213,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69215,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_df = pd.read_csv('emotion_Validation.csv')
test_df

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,이번 프로젝트에서 내가 발표 실수를 해서 우리 팀이 감점을 받아서 너무 미안해.,실수하시다니 정말 죄송한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 하려나 봐.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.,NaN,NaN
1,6,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 두려우신가봐요.,팀장님 앞에서만 있으면 떨려.,말실수 안 하는 법은 무엇이 있나요?,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,NaN,NaN
2,10,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,친구랑 나랑 같이 백수였는데 나 혼자 취업을 해서 그 친구와 멀어질까 봐 걱정이야.,친구분이랑 사이가 멀어질까 봐 걱정이시군요.,난 그 친구랑 안 멀어지고 잘 지내고 싶어.,안 멀어지고 잘 지낼 수 있는 방법은 무엇이 있을까요?,앞에서 회사 얘기는 조금하고 나도 그 친구 취업을 도와줘야겠어.,친구분도 취업하셔서 사이가 안 멀어지시면 좋겠네요.,NaN,NaN
3,18,청소년,남성,학교폭력/따돌림,해당없음,슬픔,마비된,학교에서 아이들이 날 피하는데 내가 한 말 때문인 거 같아.,학교에서 따돌림을 겪고 계시는군요.,그걸 안 이후로 아무런 생각도 나질 않아서 슬퍼.,아무런 생각도 나질 않아서 답답하겠네요. 어떻게 하면 상황이 나아질까요?,아이들을 찾아가서 내가 한 말에 대해 사과를 하면 관계 개선이 될 거 같아.,사과가 잘 받아들여져서 친구들과 원만한 관계가 되길 바랄게요.,NaN,NaN
4,27,청소년,남성,학교폭력/따돌림,해당없음,당황,외로운,친구들이 요즘 나를 따돌리고 있어. 혼자인 게 너무 슬퍼.,혼자 된 기분이 너무 슬프겠어요.,내가 한 친구랑 싸웠는데 친구들이 나랑 싸운 친구 편만 들더라.,친구들이 싸운 친구의 편만 들어서 많이 외로우셨겠어요.,나중에 그 친구에게 왜 그랬는지 자세히 들어보고 싶어.,친구와의 대화로 서로 잘 해결되길 바라요.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5125,51390,중년,남성,건강,해당없음,불안,혼란스러운,요즘 혼란스러워.,무슨 사건이 있으셨나요?,닥터가 건강하다고 하는데 몸이 안좋아.,훌륭히 이겨내실거라 믿어요.,NaN,NaN,NaN,NaN
5126,51397,청소년,남성,학교폭력/따돌림,해당없음,슬픔,슬픔,내 친구가 학교에서 따돌림을 당하고 있어.,친구때문에 근심이 많으시겠네요.,내가 도와줄수가 없어서 매우 슬퍼.,정말 마음이 아프실것 같아요.,NaN,NaN,NaN,NaN
5127,51415,청년,여성,"연애, 결혼, 출산",해당없음,상처,고립된,미국에 결혼와서 타지에 혼자 살려니 너무 외로워.,주변에 친하거 지낼만한 벗이라도 없으신가요?,여긴 완전 시골이라 나 홀로만 동양인이야. 난 잉글리쉬도 잘 못해서 고립감이 엄습한다고.,어떻게해야 외지에서의 고립감을 이겨낼 수 있을까요?,먼저 영어공부나 열심히 하라던데?,언어가 통하면 외로움도 많이 줄어들겠네요.,NaN,NaN
5128,51418,청년,여성,"진로, 취업, 직장",해당없음,상처,고립된,회사 동기가 없어서 너무 외로워.,왜 그런 마음이 들었는지 세밀히 말씀해주실 수 있나요.,모두 후배 또는 선배라서 나만 고립된 기분이야. 쓸쓸해.,속상하시겠네요. 어떻게 하면 기분이 괜찮아질 수 있을까요?,타 부서에는 내 입사 동기가 있다던데 그 분이랑 가까워져봐야겠어.,굳 아이디어네요.,NaN,NaN


In [ ]:
condition = (train_df.감정_대분류 == '불안') | (train_df.감정_대분류 =='당황') | (train_df.감정_대분류 =='슬픔') | (train_df.감정_대분류 =='분노') | (train_df.감정_대분류 =='상처') | (train_df.감정_대분류 =='불안 ')

In [ ]:
condition2 = (test_df.감정_대분류 == '불안') | (test_df.감정_대분류 =='당황') | (test_df.감정_대분류 =='슬픔') | (test_df.감정_대분류 =='분노') | (test_df.감정_대분류 =='상처') | (test_df.감정_대분류 =='불안 ')

In [ ]:
train_df2 = train_df[condition]

In [ ]:
test_df2 = test_df[condition2]
test_df2

,번호,연령,성별,상황키워드,신체질환,감정_대분류,감정_소분류,사람문장1,시스템응답1,사람문장2,시스템응답2,사람문장3,시스템응답3,사람문장4,시스템응답4
0,1,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,이번 프로젝트에서 내가 발표 실수를 해서 우리 팀이 감점을 받아서 너무 미안해.,실수하시다니 정말 죄송한 마음이 크겠어요.,내 능력이 부족한 거 같은데 그만 다녀야 하려나 봐.,능력을 올리려면 어떤 방법이 있을까요?,퇴근 후 여가에 회사 일을 더 열심히 해서 피해가 가지 않도록 해야겠어.,꼭 좋은 결과 있길 바라요.,NaN,NaN
1,6,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.,팀장님 앞에서 말실수를 할까 봐 두려우신가봐요.,팀장님 앞에서만 있으면 떨려.,말실수 안 하는 법은 무엇이 있나요?,평소에 말을 순하게 쓰려고 노력해서 앞에서 실수를 안 해야겠다.,팀장님 앞에서 실수를 안 하시길 바라요.,NaN,NaN
2,10,청년,남성,"진로, 취업, 직장",해당없음,불안,두려운,친구랑 나랑 같이 백수였는데 나 혼자 취업을 해서 그 친구와 멀어질까 봐 걱정이야.,친구분이랑 사이가 멀어질까 봐 걱정이시군요.,난 그 친구랑 안 멀어지고 잘 지내고 싶어.,안 멀어지고 잘 지낼 수 있는 방법은 무엇이 있을까요?,앞에서 회사 얘기는 조금하고 나도 그 친구 취업을 도와줘야겠어.,친구분도 취업하셔서 사이가 안 멀어지시면 좋겠네요.,NaN,NaN
3,18,청소년,남성,학교폭력/따돌림,해당없음,슬픔,마비된,학교에서 아이들이 날 피하는데 내가 한 말 때문인 거 같아.,학교에서 따돌림을 겪고 계시는군요.,그걸 안 이후로 아무런 생각도 나질 않아서 슬퍼.,아무런 생각도 나질 않아서 답답하겠네요. 어떻게 하면 상황이 나아질까요?,아이들을 찾아가서 내가 한 말에 대해 사과를 하면 관계 개선이 될 거 같아.,사과가 잘 받아들여져서 친구들과 원만한 관계가 되길 바랄게요.,NaN,NaN
4,27,청소년,남성,학교폭력/따돌림,해당없음,당황,외로운,친구들이 요즘 나를 따돌리고 있어. 혼자인 게 너무 슬퍼.,혼자 된 기분이 너무 슬프겠어요.,내가 한 친구랑 싸웠는데 친구들이 나랑 싸운 친구 편만 들더라.,친구들이 싸운 친구의 편만 들어서 많이 외로우셨겠어요.,나중에 그 친구에게 왜 그랬는지 자세히 들어보고 싶어.,친구와의 대화로 서로 잘 해결되길 바라요.,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5125,51390,중년,남성,건강,해당없음,불안,혼란스러운,요즘 혼란스러워.,무슨 사건이 있으셨나요?,닥터가 건강하다고 하는데 몸이 안좋아.,훌륭히 이겨내실거라 믿어요.,NaN,NaN,NaN,NaN
5126,51397,청소년,남성,학교폭력/따돌림,해당없음,슬픔,슬픔,내 친구가 학교에서 따돌림을 당하고 있어.,친구때문에 근심이 많으시겠네요.,내가 도와줄수가 없어서 매우 슬퍼.,정말 마음이 아프실것 같아요.,NaN,NaN,NaN,NaN
5127,51415,청년,여성,"연애, 결혼, 출산",해당없음,상처,고립된,미국에 결혼와서 타지에 혼자 살려니 너무 외로워.,주변에 친하거 지낼만한 벗이라도 없으신가요?,여긴 완전 시골이라 나 홀로만 동양인이야. 난 잉글리쉬도 잘 못해서 고립감이 엄습한다고.,어떻게해야 외지에서의 고립감을 이겨낼 수 있을까요?,먼저 영어공부나 열심히 하라던데?,언어가 통하면 외로움도 많이 줄어들겠네요.,NaN,NaN
5128,51418,청년,여성,"진로, 취업, 직장",해당없음,상처,고립된,회사 동기가 없어서 너무 외로워.,왜 그런 마음이 들었는지 세밀히 말씀해주실 수 있나요.,모두 후배 또는 선배라서 나만 고립된 기분이야. 쓸쓸해.,속상하시겠네요. 어떻게 하면 기분이 괜찮아질 수 있을까요?,타 부서에는 내 입사 동기가 있다던데 그 분이랑 가까워져봐야겠어.,굳 아이디어네요.,NaN,NaN


In [ ]:
train_df2.loc[(train_df['감정_대분류'] == "당황"), '감정_대분류'] = 0  
train_df2.loc[(train_df['감정_대분류'] == "분노"), '감정_대분류'] = 1  
train_df2.loc[(train_df['감정_대분류'] == "불안"), '감정_대분류'] = 2  
train_df2.loc[(train_df['감정_대분류'] == "상처"), '감정_대분류'] = 3  
train_df2.loc[(train_df['감정_대분류'] == "슬픔"), '감정_대분류'] = 4  
train_df2.loc[(train_df['감정_대분류'] == "불안 "), '감정_대분류'] = 2  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
test_df2.loc[(test_df2['감정_대분류'] == "당황"), '감정_대분류'] = 0  
test_df2.loc[(test_df2['감정_대분류'] == "분노"), '감정_대분류'] = 1  
test_df2.loc[(test_df2['감정_대분류'] == "불안"), '감정_대분류'] = 2  
test_df2.loc[(test_df2['감정_대분류'] == "상처"), '감정_대분류'] = 3  
test_df2.loc[(test_df2['감정_대분류'] == "슬픔"), '감정_대분류'] = 4  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [ ]:
train = []
for q, label in zip(X_train, train_df2['감정_대분류'])  :
    data = []
    data.append(q)
    data.append(str(label))

    train.append(data)

In [ ]:
test = []
for q, label in zip(X_test, test_df2['감정_대분류'])  :
    data = []
    data.append(q)
    data.append(str(label))

    test.append(data)

In [ ]:
train[0]

['당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.', '2']

In [ ]:
test[0]

['이번 프로젝트에서 내가 발표 실수를 해서 우리 팀이 감점을 받아서 너무 미안해.', '2']

In [ ]:
len(train)

34148

In [ ]:
X_train = train_df2['사람문장1'].dropna(how='any') # Null 값 제거
X_train

1                  당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.
2                  고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.
5        이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...
6                            사십 대에 접어들면서 머리카락이 많이 빠져 고민이야.
7                                           이제 돈이라면 지긋지긋해.
                               ...                        
40872    오늘 식당에서 서빙을 하는데 젊은 청년들이 반말하길래 기분이 나빠 나도 반말을 하며...
40873              최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해.
40874    같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...
40876            엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.
40878                  친구 관계가 너무 힘들어. 베푸는 만큼 돌아오지 않는 것 같아.
Name: 사람문장1, Length: 34148, dtype: object

In [ ]:
X_test = test_df2['사람문장1'].dropna(how='any') # Null 값 제거
X_test

0         이번 프로젝트에서 내가 발표 실수를 해서 우리 팀이 감점을 받아서 너무 미안해.
1            내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.
2       친구랑 나랑 같이 백수였는데 나 혼자 취업을 해서 그 친구와 멀어질까 봐 걱정이야.
3                    학교에서 아이들이 날 피하는데 내가 한 말 때문인 거 같아.
4                     친구들이 요즘 나를 따돌리고 있어. 혼자인 게 너무 슬퍼.
                             ...                      
5125                                         요즘 혼란스러워.
5126                           내 친구가 학교에서 따돌림을 당하고 있어.
5127                       미국에 결혼와서 타지에 혼자 살려니 너무 외로워.
5128                                회사 동기가 없어서 너무 외로워.
5129                              삶이 너무 외로워. 버려진 느낌이야.
Name: 사람문장1, Length: 4289, dtype: object

In [ ]:
y_train = train_df2['감정_대분류'].dropna(how='any')
y_train

1        2
2        0
5        2
6        4
7        1
        ..
40872    1
40873    3
40874    2
40876    4
40878    2
Name: 감정_대분류, Length: 34148, dtype: object

In [ ]:
y_test = test_df2['감정_대분류'].dropna(how='any')
y_test

0       2
1       2
2       2
3       4
4       0
       ..
5125    2
5126    4
5127    3
5128    3
5129    3
Name: 감정_대분류, Length: 4289, dtype: object

In [ ]:
print(y_train.unique())

[2 0 4 1 3]


In [ ]:
print(y_test.unique())

[2 4 0 1 3]


In [ ]:
import numpy as np

m = {"당황":0, "분노": 1, "불안":2, "상처":3, "슬픔":4, '불안 ':2}
y_train = y_train.map(m)
y_test = y_test.map(m)

In [ ]:
y_test.unique()

array([nan])

In [ ]:
y_train.unique()

array([nan])

In [ ]:
train = pd.DataFrame({'document':X_train,'label': y_train})
train.reset_index(inplace=True)
train = train.iloc[:, 1:]
train

,document,label
0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,2
1,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,0
2,이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...,2
3,사십 대에 접어들면서 머리카락이 많이 빠져 고민이야.,4
4,이제 돈이라면 지긋지긋해.,1
...,...,...
34143,오늘 식당에서 서빙을 하는데 젊은 청년들이 반말하길래 기분이 나빠 나도 반말을 하며...,1
34144,최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해.,3
34145,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,2
34146,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,4


In [ ]:
test = pd.DataFrame({'document':X_test,'label': y_test})
test.reset_index(inplace=True)
test = test.iloc[:, 1:]
test

,document,label
0,이번 프로젝트에서 내가 발표 실수를 해서 우리 팀이 감점을 받아서 너무 미안해.,2
1,내가 평소에 말이 좀 험한데 무서운 팀장님 앞에서 말실수할까 봐 걱정이야.,2
2,친구랑 나랑 같이 백수였는데 나 혼자 취업을 해서 그 친구와 멀어질까 봐 걱정이야.,2
3,학교에서 아이들이 날 피하는데 내가 한 말 때문인 거 같아.,4
4,친구들이 요즘 나를 따돌리고 있어. 혼자인 게 너무 슬퍼.,0
...,...,...
4284,요즘 혼란스러워.,2
4285,내 친구가 학교에서 따돌림을 당하고 있어.,4
4286,미국에 결혼와서 타지에 혼자 살려니 너무 외로워.,3
4287,회사 동기가 없어서 너무 외로워.,3


In [ ]:
len(train)

34148

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
def trans(dataset, sent_idx):
  print([i[sent_idx]] for i in dataset)

In [ ]:
T = trans(train, 0)
T

<generator object trans.<locals>.<genexpr> at 0x7f0f549bfb50>


In [ ]:
# Setting parameters
max_len = 64 
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
tok

NameError: ignored

In [ ]:
train

,document,label
0,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,2
1,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,0
2,이제 곧 은퇴할 시기가 되었어. 내가 먼저 은퇴를 하고 육 개월 후에 남편도 은퇴를...,2
3,사십 대에 접어들면서 머리카락이 많이 빠져 고민이야.,4
4,이제 돈이라면 지긋지긋해.,1
...,...,...
34143,오늘 식당에서 서빙을 하는데 젊은 청년들이 반말하길래 기분이 나빠 나도 반말을 하며...,1
34144,최 영감이 노인들에게 나를 이상한 사람으로 이야기해서 답답하고 억울해.,3
34145,같이 사는 친구가 애완견을 데려왔는데 대부분 내가 돌보고 있어. 내가 주인인가 혼란...,2
34146,엄마는 내 꿈인 작가를 응원해 주고는 했는데 지금은 안 그래. 너무 슬퍼.,4


In [ ]:
vocab

Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")

In [ ]:
tokenizer

'/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece'

In [ ]:
tok

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
train[0]

KeyError: ignored

In [ ]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(train, test_size=0.25, random_state=0)

In [ ]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      

In [ ]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [ ]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)


In [ ]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/534 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.7088931798934937 train acc 0.171875
epoch 1 batch id 201 loss 1.0774405002593994 train acc 0.34888059701492535
epoch 1 batch id 401 loss 1.1681653261184692 train acc 0.454839463840399
epoch 1 train acc 0.4852040418227216


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6204044117647058


  0%|          | 0/534 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.6435838341712952 train acc 0.75
epoch 5 batch id 201 loss 0.46308326721191406 train acc 0.7808613184079602
epoch 5 batch id 401 loss 0.6997621059417725 train acc 0.7971087905236908
epoch 5 train acc 0.797791822721598


  0%|          | 0/68 [00:00<?, ?it/s]

epoch 5 test acc 0.6240808823529411


In [ ]:
model.save(model.state_dict(), "model_kobert.pt")

AttributeError: ignored

In [ ]:
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
# 테스트 문장 예측
test_sentence = '음식을 잘못 먹었나봐요... 발열, 구토, 복통, 설사 증상이 있어요.'
test_label = 7 # 실제 질병

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['질문 내용', '질병명']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

RuntimeError: ignored

In [ ]:

# 학습용 데이터셋 불러오기
import pandas as pd
dataset_train1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/자연어처리/user_conversation.csv')
dataset_train1.head()

# 데이터 전처리
dataset_train1.drop(['Unnamed: 0', 'Unnamed: 0.1', '질문 제목', '작성 시간', '태그', 'url'], axis=1, inplace=True)
dataset_train1.head()

dataset_train1['질병명'].unique()

# 대표 질병 5개 추출
data1 = dataset_train1[dataset_train1['질병명'] == '수족냉증']
data2 = dataset_train1[dataset_train1['질병명'] == '식중독']
data3 = dataset_train1[dataset_train1['질병명'] == '소화불량']
data4 = dataset_train1[dataset_train1['질병명'] == '질염']
data5 = dataset_train1[dataset_train1['질병명'] == '비염']
new_data = data1.append([data2, data3, data4, data5], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

# 질병명 라벨링
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(new_data['질병명'])
new_data['질병명'] = encoder.transform(new_data['질병명'])
new_data.head()

# 라벨링된 질병명 매핑 ex) {0: '비염', 1: '소화불량', 2: '수족냉증', 3: '식중독', 4: '질염'}
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

# Train / Test set 분리
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))
        
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

data_train = BERTDataset(train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, max_len, True, False)

# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 5, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)
      
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    
# 테스트 문장 예측
test_sentence = '음식을 잘못 먹었나봐요... 발열, 구토, 복통, 설사 증상이 있어요.'
test_label = 7 # 실제 질병

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['질문 내용', '질병명']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)